
```
  _____                                 ____                                        ____                  _           _
 |_   _|   ___    ___   _ __    _   _  / ___|    ___   _ __  __   __   ___   _ __  / ___|    ___   _ __  (_)  _ __   | |_
   | |    / _ \  / _ \ | '_ \  | | | | \___ \   / _ \ | '__| \ \ / /  / _ \ | '__| \___ \   / __| | '__| | | | '_ \  | __|
   | |   |  __/ |  __/ | | | | | |_| |  ___) | |  __/ | |     \ V /  |  __/ | |     ___) | | (__  | |    | | | |_) | | |_
   |_|    \___|  \___| |_| |_|  \__, | |____/   \___| |_|      \_/    \___| |_|    |____/   \___| |_|    |_| | .__/   \__|
                                |___/                                                                        |_|
```
**Crea un Servidor de Minecraft en Google Colab! (Creado por TeenyDesert9892)**

---

El Script de abajo se encarga de manejar el servidor, puedes instalar una gran variedad de versiones y configurarlo a tu gusto.
Lo unico que tienes que hacer en el script de abajo es añadir el nombre del direcorio de tu servidor para tener una mayor organizacion con el servidor y ya esta.

In [ ]:
# Por favor introducir el nombre de la carpeta donde quieres que se encuentre tu servidor de minecraft

folder_name = "Minecraft Server"

# A partir de aqui no tienes que hacer nada mas

! pip install pyngrok

import glob
import json
import os
import shutil
import subprocess
from threading import Thread
from google.colab import drive

import requests
from pyngrok import conf, ngrok

folder_name.replace(" ", "-")

server_type_message = "# Escoge el tipo de servidor, estas son las opciones que estan sostenidas por este script:" \
                        "- paper (Lo mas vanilla que hay, tambien es muy similar y retrocompatible con Spigot y Bukkit)" \
                        "- spigot (Muy similar a paper pero con alguna caracteristica menos)" \
                        "- mohist (Se usa para poner mods de forge y plugins a la vez)" \
                        "- forge (Se usa para mods)" \
                        "- fabric (Es una alternativa a Forge con sus diferencias pero no esretrocompatible con forge)" \
                        "- vanilla (Minecraft Vanilla sin mods ni plugins)" \
                        ">>> "
server_version_message = "Las nuevas versiones deberian funcionar, pero no esta garantizado si salieron hace poco o hicieron cambios bruscos nunca antes vistos..." \
                            "En todo caso consulta a TeenyDesert9892" \
                            "Versiones que han sido testeadas:" \
                            "- 1.20.X" \
                            "- 1.19.X" \
                            "- 1.18.X" \
                            "- 1.17.X" \
                            "- 1.16.X" \
                            "- 1.15.X" \
                            "- 1.14.X" \
                            "- 1.13.X" \
                            "- 1.12.X" \
                            "- 1.11.X" \
                            "- 1.10.X" \
                            "- 1.9.X" \
                            "- 1.8.X" \
                            ">>> "
max_ram_message = "Cuanta ram quieres que pueda usar como maximo tu servidor? (Introducela en GB)" \
                    ">>> "
connect_service_message = "Que servicio quieres usar para el servidor:" \
                            "p - (playit) Es un servicio recomendado que solo consta de intalar su aplicacion, crearte una cuenta en su web y vincular el dispositivo." \
                            "n - (ngrok) Es un servicio recomendado por si vas a tener tu servidor abierto infinitamente por que a diferencia del otro su ip se reinicia cada vez que lo abres, este no requiere de instalar nada." \
                            ">>> "
ngrok_token_message = "Consigue tu token en: https://dashboard.ngrok.com/get-started/your-authtoken" \
                        "Introduce tu token de ngrok: "
ngrok_region_message = "Elige la region del servidor:" \
                        "- ap - Asia/Pacific (Singapore)" \
                        "- au - Australia (Sydney)" \
                        "- eu - Europa (Frankfurt - Alemania)" \
                        "- in - India (Mumbai)" \
                        "- jp - Japan (Tokyo)" \
                        "- sa - America del sur (Sao Paulo - Brasil)" \
                        "- us - United States (Ohio)" \
                        ">>> "
logo = "---------------TeenyServerScript---------------"


drive.mount("/content/drive")

os.mkdir("/content/drive/My drive")
os.chdir("/content/drive/My drive")

print("This script was made by TeenyDesert9892")


def load_config():
    jsonConfig = open("server_config.json", "r")
    global config
    config = json.load(jsonConfig)


def save_config():
    jsonConfig = open("server_config.json", "w")
    global config
    json.dump(config, jsonConfig, indent=4)


def create_server():
    server_version = config[0]["server_version"]
    server_type = config[0]["server_type"]

    try:
        subprocess.run("ls", shell=True)

        if not os.path.exists(f"{folder_name}"):
            os.mkdir(f"{folder_name}")

        print('Descargando a archivos del servidor...')

        serverURL = ""

        if server_type == 'paper':
            serverURL = "https://jar.smd.gg/download/paper/" + server_version + "/latest"

        elif server_type == 'spigot':
            if not os.path.exists("BuildTools/BuildTools.jar"):
                buildTools = requests.get("https://hub.spigotmc.org/jenkins/job/BuildTools/lastSuccessfulBuild/artifact/target/BuildTools.jar")
                if buildTools.status_code == 200:
                    if not os.path.exists("assets/BuildTools"):
                        os.mkdir("BuildTools")
                    with open("BuildTools/BuildTools.jar", "wb") as file:
                        file.write(buildTools.content)
                        file.close()
            os.chdir("BuildTools")
            subprocess.run(f"java -jar BuildTools.jar --rev " + server_version)
            os.chdir("..")
            shutil.move(f"BuildTools/spigot-{server_version}.jar", folder_name)
            os.rename(folder_name + f"/spigot-{server_version}.jar", f"{folder_name}/spigot.jar")

        elif server_type == 'mohist':
            url = requests.get("https://mohistmc.com/api/v2/projects/mohist")
            if url.status_code == 200:
                for version in str(url.content).replace("b", "").replace("{", "").replace("}", "").replace("[", "").replace("]", "").replace('"', '').replace("'", "").split(":")[1].split(","):
                    if version == server_version:
                        getBuilds = requests.get("https://mohistmc.com/api/v2/projects/mohist/" + server_version + "/builds")
                        if getBuilds.status_code == 200:
                            with open("builds.json", "wb") as file:
                                file.write(getBuilds.content)
                                file.close()
                        buildsFile = open("builds.json", "r")
                        builds = json.load(buildsFile)
                        buildsFile.close()
                        os.remove("builds.json")
                        bestBuild = 0
                        for build in builds["builds"]:
                            if build["number"] > bestBuild:
                                bestBuild = build["number"]
                                serverURL = build["url"]

        elif server_type == 'forge':
            forgeVersion = mllb.forge.find_forge_version(server_version)
            serverURL = "https://maven.minecraftforge.net/net/minecraftforge/forge/" + forgeVersion + "/forge-" + forgeVersion + "-installer.jar"

        elif server_type == 'vanilla':
            serverURL = "https://jar.smd.gg/download/vanilla/" + server_version

        elif server_type == 'fabric':
            serverURL = 'https://maven.fabricmc.net/net/fabricmc/fabric-installer/1.0.1/fabric-installer-1.0.1.jar'

        else:
            print("El tipo de version que seleccionaste no es valido")

        jar_list = {'paper': 'server.jar', 'fabric': 'fabric-installer.jar', 'mohist': 'mohist.jar', 'spigot': 'spigot.jar',
                    'generic': 'server.jar', 'forge': 'mohist.jar', 'vanilla': 'vanilla.jar', 'snapshot': 'snapshot.jar'}

        jar_name = jar_list[server_type]

        os.chdir(folder_name)

        if server_type != 'spigot':
            request = requests.get(serverURL)
            if request.status_code == 200:
                with open(f'{jar_name}', 'wb') as file:
                    file.write(request.content)
                    file.close()
            elif request.status_code == 502:
                print('Error: ' + str(request.status_code) + "! La pagina se ha caido temporalmente prueba mas tarde...")
            else:
                print('Error: ' + str(request.status_code) + '! La versión que has elegido probablemente no funciona / Puede ser que hallas introducido una version inexistente. Prueba a ejecutar el codigo otra vez por si acaso fue un pequeño error.')

        if server_type == 'fabric':
            subprocess.run(f"java -jar fabric-installer.jar server --mcversion ${server_version} --downloadMinecraft", shell=True)
            minecraft_server_base = requests.get("https://jar.smd.gg/download/vanilla/" + server_version)
            os.remove("fabric-installer.jar")
            if minecraft_server_base.status_code == 200:
                with open("server.jar", "wb") as jar:
                    jar.write(minecraft_server_base.content)
                    jar.close()

        if server_type == 'forge':
            subprocess.run("java -jar forge.jar --installServer", shell=True)

        print("Completado!")
        subprocess.run("echo eula=true>> eula.txt", shell=True)

        print("\n____________________AVISO!!!____________________\nSe ha creado un nuevo archivo al lado de TeenyServerScript llamado server_config.json\nSe recomienda mirarlo antes de iniciar el servidor por primera vez para saber que contiene\nCualquer cambio puede dañar al funcionamiento del servidor asi que sea delicado")
    except:
        if os.path.exists(folder_name):
            shutil.rmtree(folder_name)

        if os.path.exists("server_config.json"):
            os.remove("server_config.json")

        print("\nAlguno de los datos que has introducido es invalido\nDeteniendo el script...")


def start_server():
    server_type = config[0]["server_type"]
    max_ram = config[0]["max_ram"]
    connect_service = config[1]["connect_service"]
    ngrok_token = config[1]["ngrok_token"]
    ngrok_region = config[1]["ngrok_region"]

    if os.path.exists(f"{folder_name}"):
        os.chdir(f"{folder_name}")

        jar_list = {'paper': 'server.jar', 'fabric': 'fabric-server-launch.jar', 'mohist': 'mohist.jar', 'spigot': 'spigot.jar',
                    'generic': 'server.jar', 'forge': 'forge.jar', 'vanilla': 'vanilla.jar', 'snapshot': 'snapshot.jar'}

        jar_name = jar_list[server_type]

        if server_type == "paper":
            server_flags = "-XX:+UseG1GC -XX:+ParallelRefProcEnabled -XX:MaxGCPauseMillis=200 -XX:+UnlockExperimentalVMOptions -XX:+DisableExplicitGC -XX:+AlwaysPreTouch -XX:G1NewSizePercent=30 -XX:G1MaxNewSizePercent=40 -XX:G1HeapRegionSize=8M -XX:G1ReservePercent=20 -XX:G1HeapWastePercent=5 -XX:G1MixedGCCountTarget=4 -XX:InitiatingHeapOccupancyPercent=15 -XX:G1MixedGCLiveThresholdPercent=90 -XX:G1RSetUpdatingPauseTimePercent=5 -XX:SurvivorRatio=32 -XX:+PerfDisableSharedMem -XX:MaxTenuringThreshold=1 -Dusing.aikars.flags=https://mcflags.emc.gs -Daikars.new.flags=true"
        else:
            server_flags = ""
        memory_allocation = f"-Xms512M {max_ram}"

        subprocess.run("clear", shell=True)

        if connect_service == "n":
            print('\nIniciando servidor con ngrok...')

            subprocess.run(f"ngrok authtoken {ngrok_token}", shell=True)
            conf.get_default().region = ngrok_region

            urlj = ngrok.connect('25565', 'tcp')
            print('La IP de tu servidor de java es ' + ((str(urlj).split('"')[1::2])[0]).replace('tcp://', ''))
            if os.path.exists(str(f"{os.getcwd()}/plugins/Geyser-Spigot.jar")):
                urlb = ngrok.connect('19132', 'tcp')
                print('La IP de tu servidor e bedrock es ' + ((str(urlb).split('"')[1::2])[0]).replace('tcp://', ''))

            launch_server(memory_allocation, server_flags, jar_name)

        elif connect_service == "p":
            print('\nIniciando servidor con playit...')

            subprocess.run("curl -SsL https://playit-cloud.github.io/ppa/key.gpg | sudo apt-key add -", shell=True)
            subprocess.run("sudo curl -SsL -o /etc/apt/sources.list.d/playit-cloud.list https://playit-cloud.github.io/ppa/playit-cloud.list", shell=True)
            subprocess.run("sudo apt update &>/dev/null && sudo apt install playit &>/dev/null && echo 'Playit.gg instalado' || echo 'Error al instalar playit'", shell=True)

            def playit():
                subprocess.run("playit")

            playitThread = Thread(target=playit, daemon=True)
            playitThread.start()
            
            launch_server(memory_allocation, server_flags, jar_name)
        else:
            print("Servicio desconocido, el servidor no iniciara")
    else:
        print("\nAun no tienes ningun servidor creado")


def launch_server(memory_allocation=str, server_flags=str, jar_name=str):
    server_type = config[0]["server_type"]
    server_version = config[0]["server_version"]

    if server_type == "forge":
        if float(server_version.replace(".", "", 1)) < 117:
            oldpathtoforge = glob.glob(f"forge-{server_version}-*.jar")
            if oldpathtoforge:
                path = oldpathtoforge[0]
                subprocess.run(f"java {memory_allocation} -jar {path} nogui", shell=True)
            else:
                print("No ha sido encontrado 'forge universal jar'")
        else:
            pathtoforge = glob.glob(f"libraries/net/minecraftforge/forge/{server_version}-*/unix_*.txt")
            if pathtoforge:
                path = pathtoforge[0]
                subprocess.run(f"java @user_jvm_args.txt @{path} $@ nogui", shell=True)
            else:
                print("No ha sido encontrado 'unix_args.txt'")
    else:
        subprocess.run(f"java {memory_allocation} {server_flags} -jar {jar_name} nogui", shell=True)


def create_server_menu():
    server_type = input(f"\n{server_type_message} ")
    server_version = input(f"\n{server_version_message} ")
    max_ram = "-Xmx"+input(f"\n{max_ram_message} ")+"G"
    connect_service = input(f"\n{connect_service_message} ")

    if connect_service == "n":
        ngrok_token = input(f"\n{ngrok_token_message} ")
        ngrok_region = input(f"\n{ngrok_region_message} ")

    elif connect_service == "p" or connect_service == "l":
        ngrok_token = ""
        ngrok_region = "eu"
    else:
        print("\nNo se especifico el servicio de manera correcta")

    config[0]["server_type"] = server_type
    config[0]["server_version"] = server_version
    config[0]["max_ram"] = max_ram
    config[1]["connect_service"] = connect_service
    config[1]["ngrok_token"] = ngrok_token
    config[1]["ngrok_region"] = ngrok_region

    save_config()

    subprocess.run("clear", shell=True)

    create_server()


def configure_options():
    if os.path.exists("server_config.json"):
        option = input("Select the option you want to edit:\n(1) Change server Ram\n(2) Change service type\n(3) Delete current server")
        if option == "1":
            config[0]["max_ram"] = "-Xmx"+input(f'\n {max_ram_message} ')+"G"
        elif option == "2":
            config[1]["connect_service"] = input(f"\n {connect_service_message} ")
            if config[1]["connect_service"] == "n":
                config[1]["ngrok_token"] = input(f"\n{ngrok_token_message} ")
                config[1]["ngrok_region"] = input(f"\n{ngrok_region_message} ")
            elif config[1]["connect_service"] == "p" or config[1]["connect_service"] == "l":
                config[1]["ngrok_token"] = ""
                config[1]["ngrok_region"] = "eu"
            else:
                print("\nNo se especifico el servicio de manera correcta")
        elif option == "3":
            confirm_delete = input("Estas seguro de que quieres eliminar el servidor? (y/n): ").lower()
            if confirm_delete == "y":
                shutil.rmtree(folder_name)
                os.remove("server_config.json")
        else:
            print("\nOpcion no valida")
        save_config()
    else:
        print("No has creado ningun servidor aun")


if __name__ == "__main__":
    try:
        load_config()
    except:
        config = [{"server_type": "", "server_version": "", "max_ram": ""}, {"connect_service": "", "ngrok_token": "", "ngrok_region": ""}]

    print(logo)
    selection = input("\nOpciones para iniciar el servidor:\n(1) Instalar un servidor\n(2) Inicar el servidor (primero usa el 1)\n(3) Configurar opciones del servidor\n(4) Detener el programa\n>>> ")

    subprocess.run("clear", shell=True)

    if selection == "1":
        create_server_menu()
    elif selection == "2":
        start_server()
    elif selection == "3":
        configure_options()
    elif selection == "4":
        executing = False
    else:
        print("\nSeleccion no valida")

**IMPORTANTE - COSAS QUE TIENES QUE SABER**

Google Colab no esta pensado para crear servidores de Minecraft, pero se pueden hacer. Google Colab promete funcionar 12 horas seguidas, sin embargo es posible que se cierre si no hay una persona que este revisando la página o usando la consola, basicamente si la página detecta inactividad el proceso terminará. El rendimiento es de 12 GB de RAM aproximadamente, junto con un procesador con una potencia de 2.2 GHz y 2 hilos, esto es mejor que muchos hostings de pago, pero no tiene soporte y no promete estar abierto siempre, se pueden poner mods, pero no es recomendable saturarlo con muchos al igual que plugins, lo que más limita a este servidor es su procesador el cuál, no es muy bueno, cualquier hosting por 12€ puede ofrecerte un servidor de mayor calidad, si tienes pensado crear algo para muchos usuarios, es mucho mejor eso. Los servidores utilizados son los de Google junto con el proxy de Ngrok o Playit, depende tu eleccion.